# Many Model Training with Ray Tune

This template is a quickstart to using [Ray Tune](https://docs.ray.io/en/latest/tune/index.html) for batch inference. Ray Tune is one of many libraries under the [Ray AI Runtime](https://docs.ray.io/en/latest/ray-air/getting-started.html). See [this blog post](https://www.anyscale.com/blog/training-one-million-machine-learning-models-in-record-time-with-ray) for more information on the benefits of performing many model training with Ray!

This template walks through time-series forecasting using `statsforecast`, but the framework and data format can be swapped out easily -- they are there just to help you build your own application!

At a high level, this template will:

1. [Define the training function for a single partition of data.](https://docs.ray.io/en/latest/tune/tutorials/tune-run.html)
2. [Define a Tune search space to run training over many partitions of data.](https://docs.ray.io/en/latest/tune/tutorials/tune-search-spaces.html)
3. [Extract the best model per dataset partition from the Tune experiment output.](https://docs.ray.io/en/latest/tune/examples/tune_analyze_results.html)

> Slot in your code below wherever you see the ✂️ icon to build a many model training Ray application off of this template!

In [ ]:
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS
from pyarrow import parquet as pq
from sklearn.metrics import mean_squared_error

import ray
from ray import tune
from ray.air import session


> ✂️ Replace this value to change the number of data partitions you will use. This will be total the number of Tune trials you will run!
>
> Note that this template fits two models per data partition and reports the best performing one.
>
> Try starting with the recommended value in the table below:

| Template Size | `NUM_DATA_PARTITIONS` |
| ------------- | --------------------- |
|Small-scale (single-node) | 50  |
|Large-scale (multi-node)  | 1000 |

In [ ]:
NUM_DATA_PARTITIONS: int = 50


```{tip}
If you're running the small-scale version of the template, try setting
the number of trials to the recommended number of trials for the large-scale version.
It'll be much slower, but you'll see the dramatic speedup once distributing the load
to a multi-node Ray cluster in the large-scale version!
```

> ✂️ Replace the following with your own data-loading and evaluation helper functions. (Or, just delete these!)

In [ ]:
def get_m5_partition(unique_id: str) -> pd.DataFrame:
    df = (
        pq.read_table(
            "s3://anonymous@m5-benchmarks/data/train/target.parquet",
            columns=["item_id", "timestamp", "demand"],
            filters=[("item_id", "=", unique_id)],
        )
        .to_pandas()
        .rename(columns={"item_id": "unique_id", "timestamp": "ds", "demand": "y"})
    )
    df["unique_id"] = df["unique_id"].astype(str)
    df["ds"] = pd.to_datetime(df["ds"])
    return df.dropna()


def evaluate_cross_validation(df, metric):
    models = df.drop(columns=["ds", "cutoff", "y"]).columns.tolist()
    evals = []
    for model in models:
        eval_ = (
            df.groupby(["unique_id", "cutoff"])
            .apply(lambda x: metric(x["y"].values, x[model].values))
            .to_frame()
        )
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(["unique_id"]).mean(numeric_only=True)
    evals["best_model"] = evals.idxmin(axis=1)
    return evals


> ✂️ Replace this with your own training logic.

In [ ]:
model_classes = [AutoARIMA, AutoETS]
n_windows = 1


def train_fn(config: dict):
    data_partition_id = config["data_partition_id"]
    train_df = get_m5_partition(data_partition_id)

    models = [model_cls() for model_cls in model_classes]
    forecast_horizon = 4

    sf = StatsForecast(
        df=train_df,
        models=models,
        freq="D",
        n_jobs=n_windows * len(models),
    )
    cv_df = sf.cross_validation(
        h=forecast_horizon,
        step_size=forecast_horizon,
        n_windows=n_windows,
    )

    eval_df = evaluate_cross_validation(df=cv_df, metric=mean_squared_error)
    best_model = eval_df["best_model"][data_partition_id]
    forecast_mse = eval_df[best_model][data_partition_id]

    # Report the best-performing model and its corresponding eval metric.
    session.report({"forecast_mse": forecast_mse, "best_model": best_model})


trainable = train_fn
trainable = tune.with_resources(
    trainable, resources={"CPU": len(model_classes) * n_windows}
)


```{note}
`tune.with_resources` is used at the end to specify the number of resources to assign *each trial*.
Feel free to change this to the resources required by your application! You can also comment out the `tune.with_resources` block to assign `1 CPU` (the default) to each trial.

Note that this is purely for Tune to know how many trials to schedule concurrently -- setting the number of CPUs does not actually enforce any kind of resource isolation!
```

> ✂️ Replace this with your desired hyperparameter search space!
>
> For example, this template searches over the data partition ID to train a model on.

In [ ]:
data_partitions = list(pd.read_csv("item_ids.csv")["item_id"])
param_space = {
    "data_partition_id": tune.grid_search(data_partitions[:NUM_DATA_PARTITIONS]),
}


Run many model training using Ray Tune!

In [ ]:
tuner = tune.Tuner(trainable, param_space=param_space)
result_grid = tuner.fit()


> ✂️ Replace the metric and mode below with the metric you reported in your training function.

In [ ]:
sample_result = result_grid[0]
sample_result.metrics
